In [ ]:
#| default_exp websocket_rosbridge

# Websocket with rosbridge
    this notebook is a tutorial for no ros computer to get ros information by rosbridge.
    let user to know how to use roslibpy library to communicate to ros device.

In [ ]:
#| hide 
from nbdev.showdoc import *

## Function define part

In [ ]:
# |hide
import threading

In [ ]:
# |export
import roslibpy
import time

In [ ]:
# |export 
class ros_socket():
    def __init__(self, ip, port=9090):
        '''
        __init__
        
        Input:
            ip(type: string) ip address you want to connect
            port(type: int) default is 9090 
        '''
        self.ip = ip
        self.port = port
        self.topic = []
        self.node = []
        self.topic_name = ''
        self.topic_type = ''
        self.client = roslibpy.Ros(host = ip, port = port)
        self.client.run()
    
    def get_topic(self):
        self.topic = self.client.get_topics()
        return self.topic
    
    def get_node(self):
        self.node = self.client.get_nodes()
        return self.node
    
    def check_connecting(self):
        print('Is ROS connected?',  self.client.is_connected)
    
    def subscriber(self, topic_name, subscribe_callback, rate_in_ms=1000):
        '''
        subscriber
        subscribe topic with rate (default = 1sec)
        
        Input:
            topic_name(type:)
            subscribe_callback(message)
            (type: function) *only one argument message-> that will load with data you subscribing
        '''
        self.topic_name = topic_name
        self.topic_type = self.client.get_topic_type(self.topic_name)
        listener = roslibpy.Topic(self.client, self.topic_name, self.topic_type,throttle_rate = rate_in_ms)
        listener.subscribe(subscribe_callback)
    
    def publisher(self, topic_name, topic_type):
        '''
        publisher
        publish message_data to topic_name
        
        Input:
            topic_name(type:string)
            topic_type(type:)
            message_data(type:topic_type)
        '''
        talker = roslibpy.Topic(self.client, topic_name, topic_type)
        return talker
    
    def pub(self, talker, data):
        talker.publish(data)
    
    
    def println(self, ros_list):
        if len(ros_list) == 0:
            print('Empty')
        else:
            for i in ros_list:
                print(i)
        
        

## Testing Example
    you need open the rosbridge in server computer you want to linked first!

In [ ]:
socket = ros_socket('140.113.148.77',9090)
socket.check_connecting()

In [ ]:
node = socket.get_node()
socket.println(node)

In [ ]:
topic = socket.get_topic()
socket.println(topic)

In [ ]:
sensortower_status = {}

def sensor_callback(message):
   name_id = message['data'].split('@')[0]
   sensortower_status[name_id] =  message['data'].split('@')[1].split(':')
    print(sensortower_status)
socket.subscriber('/health/sensortower1', sensor_callback, 1000)

## Testing example of multi-sensor tower health

In [ ]:
import threading
import time
thread = []


def sensor_check(ip, sensor_tower_id):
    socket_sensor = ros_socket(ip, 9090)
    socket_sensor.subscriber('/health/sensortower' + str(sensor_tower_id+1), sensor_callback, 100)

for i in range(3):
    thread.append(threading.Thread(target=sensor_check, args =('140.113.148.77', i)))
    thread[i].start()

while True:
    print(sensortower_status, end = '\r')
    time.sleep(1)
    

## Testing example of publisher
If without multi-thread you can't use publisher & subscriber or multi-publisher multi-subscriber at the same time

*if you want a whle example of publisher go to see RSA2022/02_websocket_of_control_locobot

In [ ]:
publisher_tester = ros_socket('0.0.0.0', 9090)

In [ ]:
data = {'linear':{'x':0.0, 'y':0.0, 'z':0.0},'angular':{'x':0.0, 'y':0.0, 'z':0.0}}
puber = publisher_tester.publisher('/test_publish','geometry_msgs/Twist')
for i in range(100):
    publisher_tester.pub(puber, data)
    time.sleep(0.5)